In [80]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen


In [81]:
# Reading CSV File for first Batch 0-5000

match_logs_list_urls = pd.read_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/match_logs_list_urls.csv')

In [82]:
# This one works

match_logs_list_urls = list(match_logs_list_urls['0'])
match_logs_list_urls = ['https://fbref.com' + i for i in match_logs_list_urls]


In [83]:
# Pull all match_log_lists_x tables. We will convert each list individually WORK IN PROCESS

def create_match_logs_tables(match_logs_list_urls_x):

    df_30_columns = pd.DataFrame([])
    df_39_columns = pd.DataFrame([])


    for player in match_logs_list_urls_x:
        try: # this may fix "HTTP Error 404: Not Found"
            # urlopen(player)

            new_table = pd.read_html(player)[0]
            new_table.columns = new_table.columns.droplevel()
            new_table['name'] = player.split('/')[-1].replace("-Match-Logs", "")
            
            if new_table.shape[1] == 30:
                df_30_columns = df_30_columns.append(new_table, ignore_index=True)
                
            if new_table.shape[1] == 39:
                df_39_columns = df_39_columns.append(new_table, ignore_index=True)
            
        except:
            pass

    return df_30_columns, df_39_columns

In [84]:
len(match_logs_list_urls)

84959

In [85]:
df_30_columns_1, df_39_columns_1 = create_match_logs_tables(match_logs_list_urls[0:5000])

In [46]:
print(df_39_columns_1.columns)
print(df_30_columns_1.columns)

Index(['level_0', 'index', 'Date', 'Day', 'Comp', 'Round', 'Venue', 'Result',
       'Squad', 'Opponent', 'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt',
       'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Press', 'Tkl', 'Int', 'Blocks',
       'xG', 'npxG', 'xA', 'SCA', 'GCA', 'Cmp', 'Att', 'Cmp%', 'Prog',
       'Carries', 'Prog', 'Succ', 'Att', 'Match Report', 'name'],
      dtype='object')
Index(['level_0', 'index', 'Date', 'Day', 'Comp', 'Round', 'Venue', 'Result',
       'Squad', 'Opponent', 'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt',
       'Sh', 'SoT', 'CrdY', 'CrdR', 'Fls', 'Fld', 'Off', 'Crs', 'TklW', 'Int',
       'OG', 'PKwon', 'PKcon', 'Match Report', 'name'],
      dtype='object')


In [79]:
#Combining Df_30_columns_1 and df_39_columns_1 to dataframe_1
# df = df_39_columns_1 + df_30_columns_1
# dataframe_final_1 = pd.concat([df_39_columns_1, df_30_columns_1], axis=1)

cols = ['level_0', 'index', 'Date', 'Day', 'Comp', 'Round', 'Venue', 'Result',
       'Squad', 'Opponent', 'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt',
       'Sh', 'SoT', 'CrdY', 'CrdR', 'Match Report', 'name']

df_final = df_39_columns_1
# df_39_columns_1.columns = df_39_columns_1.columns.astype(str)
# df_30_columns_1.columns = df_30_columns_1.columns.astype(str)

# df_new = df_39_columns_1[['Date', 'Day']]
# df_new
# df_final = pd.concat([df_39_columns_1, df_30_columns_1], axis=1)
df_final = df_final.merge(df_30_columns_1, left_on=[cols], right_on=[cols], how='right')
# df_final.columns


KeyError: [['level_0', 'index', 'Date', 'Day', 'Comp', 'Round', 'Venue', 'Result', 'Squad', 'Opponent', 'Start', 'Pos', 'Min', 'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Match Report', 'name']]

In [67]:
df_final.shape

(126365, 73)

In [ ]:
# export to google drive as dataframe_1

pd.DataFrame(match_logs_list_urls).to_csv('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/dataframe_1.csv')